**Load data**

Load from csv into pandas. Each sequence, one by one.

In [10]:
import pandas as pd
from pathlib import Path

data_dir = Path('data')
sequences = []
for path in data_dir.iterdir():
    print('Reading: ', path)
    dataframe = pd.read_csv(path)
    sequences.append(dataframe)

Reading:  data\AAPL_data.csv
Reading:  data\AMD_data.csv
Reading:  data\APD_data.csv
Reading:  data\HST_data.csv
Reading:  data\MCD_data.csv
Reading:  data\MRO_data.csv
Reading:  data\NKE_data.csv
Reading:  data\SLB_data.csv
Reading:  data\VZ_data.csv
Reading:  data\WMT_data.csv


**Model**

Initialize Model

**Training**

Run training.